## This is a notebook which takes the two files (Trips and Leavetimes) created from the bash script that separates one route

In [1]:
import pandas as pd
import os, sys


#reading in data from trips
file_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(file_dir)
newPath = os.path.join(parent_dir, 'bus_lines/bus_66_trips.csv')


dftrips = pd.read_csv(newPath,  delimiter=',', index_col=False)


In [2]:
dftrips.tail(5)

,DayOfService,TripID,LineID,RouteID,Direction,PlannedTime_Dep,PlannedTime_Arr,ActualTime_Dep,ActualTime_Arr,DayOfServiceTripID
11555,2017-06-30,5116922,66,66_11,1,42585,38700,43195.0,38696.0,2017-06-305116922
11556,2017-06-30,5116923,66,66_13,2,46991,43500,47769.0,43386.0,2017-06-305116923
11557,2017-06-30,5116930,66,66_11,1,76631,72900,76452.0,72894.0,2017-06-305116930
11558,2017-06-30,5116931,66,66_13,2,80863,78300,81322.0,78260.0,2017-06-305116931
11559,2017-06-30,5117541,66,66_13,2,61658,57900,62376.0,57881.0,2017-06-305117541


In [3]:
#reading data from leavetimes
file_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(file_dir)
newPath = os.path.join(parent_dir, 'bus_lines/bus_66_lt.csv')


dfleavetimes = pd.read_csv(newPath, delimiter=',', index_col=False)

In [4]:
dfleavetimes.head(5)

,DayOfService,TripID,ProgrNumber,StopPointID,PlannedTime_Arr,PlannedTime_Dep,ActualTime_Arr,ActualTime_Dep,Unnamed: 8,DayOfServiceTripID
0,2017-01-03,4084264,19,2201,65540,65540,64669,64669,NaN,2017-01-034084264
1,2017-01-03,4084264,24,2215,65870,65870,65054,65054,NaN,2017-01-034084264
2,2017-01-03,4084264,29,2219,66160,66160,65354,65364,NaN,2017-01-034084264
3,2017-01-03,4084264,34,3887,66429,66429,65631,65631,NaN,2017-01-034084264
4,2017-01-03,4084264,39,3955,66721,66721,65886,65886,NaN,2017-01-034084264


In [5]:
dfleavetimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349291 entries, 0 to 349290
Data columns (total 10 columns):
DayOfService          349291 non-null object
TripID                349291 non-null int64
ProgrNumber           349291 non-null int64
StopPointID           349291 non-null int64
PlannedTime_Arr       349291 non-null int64
PlannedTime_Dep       349291 non-null int64
ActualTime_Arr        349291 non-null int64
ActualTime_Dep        349291 non-null int64
Unnamed: 8            386 non-null float64
DayOfServiceTripID    349291 non-null object
dtypes: float64(1), int64(7), object(2)
memory usage: 26.6+ MB


### Merging dataframes

trips dataframe is joined onto leavetimes dataframe where tripid and dayofservice match. They are merged using an left join. 

In [6]:
df = pd.merge(dfleavetimes, dftrips, how='left', on=['TripID', 'DayOfService'], suffixes=('_leave', '_trips'))

In [7]:
print("Trips shape:")
dftrips.shape

Trips shape:


(11560, 10)

In [8]:
print("Leavetimes shape:")
dfleavetimes.shape

Leavetimes shape:


(349291, 10)

In [9]:
df.shape

(349291, 18)

### Reading in Weather Data

In [10]:
file_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(file_dir)
newPath = os.path.join(parent_dir, '/home/student/data/weather/2017weatherClean.csv')


dfW17 = pd.read_csv(newPath, delimiter=',')

In [11]:
#Only keeping the columns that we want
dfW17  = dfW17[['dt','temp','pressure','humidity','wind_speed','weather_description','clouds_all','wind_deg']]

### Creating a column called duration which will be our target value.

In [12]:
#We want to look at only one direction
df = df.loc[(df['Direction'] == 1)]

In [13]:
#combining DayOfService and TripID into one column 
df['comb'] = df['DayOfService'].map(str) + df['TripID'].map(str)
u = df['comb'].unique()

In [14]:
len(u)

3656

In [15]:
import numpy as np
trip_66 = df['TripID'].unique()
date_66 = df['DayOfService'].unique()

result = pd.DataFrame(columns = ['dt','dayofweek','month','day','arrive_time','start_point','end_point','duration'])


# for trip in trip_46A:
#     for date in date_46A:
for date_trip in u:
        day_trip = df.loc[(df['comb'] == date_trip)].sort_values(by=['ProgrNumber'])
        try:
            day_trip.index = range(len(day_trip))

            # Calculate the time between each pair stops
            time = day_trip['ActualTime_Arr_leave'].shift(-1) - day_trip['ActualTime_Arr_leave']
            time.drop(time.tail(1).index,inplace=True)

            # Get the month and the day, month and the day of week
            month = pd.to_datetime(day_trip['DayOfService']).dt.month
            day = pd.to_datetime(day_trip['DayOfService']).dt.day
            dayofweek = pd.to_datetime(day_trip['DayOfService']).dt.dayofweek

            # Add datetime and actual_arrive time as current unix time
            datetime = pd.DatetimeIndex(day_trip['DayOfService']).astype(np.int64)/1000000000 + day_trip['ActualTime_Arr_leave']
            datetime = pd.DataFrame(data={'unixtime':datetime})

            # Conver float to int, this is for the following merge operation
            datetime['unixtime'] = datetime['unixtime'].astype(np.int64)

            # Set End point 
            end_point = day_trip['StopPointID'].shift(-1)
            end_point.drop(end_point.tail(1).index,inplace=True)

            # Merge these columns
            a = pd.concat([datetime,dayofweek, month, day, day_trip['ActualTime_Arr_leave'], day_trip['StopPointID'], end_point, time], axis = 1)
            # Change the name of columns
            a.columns = ['dt','dayofweek','month','day','arrive_time','start_point','end_point','duration']
            a.drop(a.tail(1).index,inplace=True)
            # Merge two tables
            r = pd.merge_asof(a, dfW17, on = "dt",direction='nearest')

            result = pd.concat([r, result])
            
        
        except:
            pass
        
# result

/home/student/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [16]:
result.shape

(180589, 15)

In [17]:
result.head(10)

,arrive_time,clouds_all,day,dayofweek,dt,duration,end_point,humidity,month,pressure,start_point,temp,weather_description,wind_deg,wind_speed
0,72894,75.0,30,4,1498853694,64.0,493.0,76.0,6,1016.0,7391,286.72,broken clouds,320.0,6.0
1,72958,75.0,30,4,1498853758,55.0,494.0,76.0,6,1016.0,493,286.72,broken clouds,320.0,6.0
2,73013,75.0,30,4,1498853813,97.0,495.0,76.0,6,1016.0,494,286.72,broken clouds,320.0,6.0
3,73110,75.0,30,4,1498853910,124.0,400.0,76.0,6,1016.0,495,286.72,broken clouds,320.0,6.0
4,73234,75.0,30,4,1498854034,26.0,346.0,76.0,6,1016.0,400,286.72,broken clouds,320.0,6.0
5,73260,75.0,30,4,1498854060,210.0,317.0,76.0,6,1016.0,346,286.72,broken clouds,320.0,6.0
6,73470,75.0,30,4,1498854270,123.0,312.0,76.0,6,1016.0,317,286.72,broken clouds,320.0,6.0
7,73593,75.0,30,4,1498854393,81.0,1444.0,76.0,6,1016.0,312,286.72,broken clouds,320.0,6.0
8,73674,75.0,30,4,1498854474,99.0,1445.0,76.0,6,1016.0,1444,286.72,broken clouds,320.0,6.0
9,73773,75.0,30,4,1498854573,183.0,7078.0,76.0,6,1016.0,1445,286.72,broken clouds,320.0,6.0


#### This code to get duration is EXTREMELY time consuming so we save to csv file

In [21]:
result.to_csv('/home/student/data_analytics/bus_lines/result_of_66-dir1.csv')

In [22]:
import pandas as pd
import os, sys


#reading in data
file_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(file_dir)
newPath = os.path.join(parent_dir, 'bus_lines/result_of_66-dir1.csv')


df = pd.read_csv(newPath,  delimiter=',', index_col=False)

#### Creating a file of only the columns we want and changing categorical features to binary so that they can be used by the model

The columns selected for the features was deciding in the 46A notebook file

In [26]:
#numeric features chosen for model [see 46A notebook for feature selection]
continuous = ['temp','clouds_all','wind_speed','wind_deg','pressure','humidity']
df_continouos = df[continuous]

#binary coding of start/end stops
binary_start_stops = pd.get_dummies(df['start_point'])
binary_end_stops = pd.get_dummies(df['end_point'])

#binary coding of weather
binary_weather = pd.get_dummies(df['weather_description'])

#binary Coding of dayofweek
binary_dayofweek = pd.get_dummies(df['dayofweek'])

#Binary Coding of arrive time
binary_arrivetime = pd.get_dummies(df['arrive_time'].divide(1800).round())

r = pd.concat([df['duration'], df_continouos, binary_end_stops, binary_weather, binary_start_stops, binary_dayofweek, binary_arrivetime], axis=1)

#### Finding outliers

In [27]:
sort_by_duation = r.sort_values(by='duration',ascending=False)
outliers = sort_by_duation.loc[sort_by_duation['duration'] > 3000]
outliers

,duration,temp,clouds_all,wind_speed,wind_deg,pressure,humidity,312.0,317.0,346.0,...,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0
125984,9088.0,280.15,75.0,4.0,120.0,994.0,93.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57109,6286.0,286.15,75.0,6.0,300.0,1035.0,82.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
59251,6194.0,284.73,75.0,4.0,110.0,1033.0,71.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
171404,6070.0,283.15,40.0,7.0,280.0,1010.0,76.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
151029,5313.0,272.64,75.0,3.0,270.0,1027.0,100.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
36965,5292.0,286.15,20.0,3.0,40.0,996.0,76.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
50040,4480.0,284.15,75.0,5.0,80.0,1021.0,81.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
29652,4375.0,296.17,20.0,7.0,130.0,1020.0,56.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29517,4013.0,295.72,75.0,4.0,160.0,1011.0,46.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
131056,3789.0,279.15,75.0,6.0,290.0,992.0,87.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#dropping outliers
r = r[r.duration < 3000]

In [29]:
r.to_csv('/home/student/data_analytics/bus_lines/result_of_66-dir1Model.csv')